# Trabajo Práctico 2: Predicción de tormenta en base de Datos Meteorológicos

### 1. Preparación de Datos:

#### 1.1. Lea el dataset limpio que se guardó en el TP1 (punto 2 de conclusiones)

In [1]:
# Importamos librerías a usar

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Importamos y leemos el dataset limpio del TP1

df = pd.read_csv("weatherAUS_limpio.csv")
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Cloud9am,Cloud3pm,Temp_media,amp_viento
0,2008-12-02,Albany,14.7,21.0,0.0,5.4,9.1,W,17.0,NE,...,1012.8,1009.3,17.8,19.1,0.0,0.0,2.0,6.0,17.85,11.0
1,2008-12-03,Albany,16.5,24.0,0.2,4.8,10.0,W,17.0,ENE,...,999.8,998.7,20.0,21.0,0.0,1.0,7.0,7.0,20.25,7.0
2,2008-12-04,Albany,14.1,20.6,7.8,7.0,7.2,W,17.0,N,...,1006.7,1008.5,19.8,18.0,1.0,0.0,5.0,7.0,17.35,28.0
3,2008-12-05,Albany,13.0,17.8,0.0,3.6,4.9,W,17.0,SW,...,1015.3,1016.3,16.0,17.0,0.0,0.0,3.0,5.0,15.40,6.0
4,2008-12-06,Albany,14.8,18.5,0.0,3.8,8.7,W,17.0,ESE,...,1020.1,1017.9,16.9,17.7,0.0,0.0,8.0,3.0,16.65,9.0


#### 1.2. Realice una codificación adecuada de variables categóricas si es necesario, utilizando técnicas como One-Hot Encoding por ejemplo.

In [3]:
# Convertimos la variables

df["Date"] = pd.to_datetime(df["Date"])
df["timestamp"] = pd.to_datetime(df["Date"]).astype(np.int64) / 10**9

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day

df[["Date", "Year", "Month", "Day", "timestamp"]].head()

,Date,Year,Month,Day,timestamp
0,2008-12-02,2008,12,2,1.228176e+09
1,2008-12-03,2008,12,3,1.228262e+09
2,2008-12-04,2008,12,4,1.228349e+09
3,2008-12-05,2008,12,5,1.228435e+09
4,2008-12-06,2008,12,6,1.228522e+09


In [4]:
# Convertimos las variables RainToday y RainTomorrow en booleanas

df["RainToday"] = df["RainToday"].apply(lambda x: True if x == 1 else False)
df["RainTomorrow"] = df["RainTomorrow"].apply(lambda x: True if x == 1 else False)

df[["RainToday", "RainTomorrow"]].head()

,RainToday,RainTomorrow
0,False,False
1,False,True
2,True,False
3,False,False
4,False,False


In [5]:
# Identificamos las variables categóricas

variables_categoricas = df.select_dtypes(include=["object"]).columns
variables_categoricas

Index(['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], dtype='object')

In [6]:
# Aplicamos el One-Hot Encoding a las variables categóricas

df_encoded = pd.get_dummies(df, columns=variables_categoricas)
df_encoded.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,2008-12-02,14.7,21.0,0.0,5.4,9.1,17.0,6.0,17.0,64.0,...,False,False,False,False,False,False,False,False,False,False
1,2008-12-03,16.5,24.0,0.2,4.8,10.0,17.0,19.0,24.0,36.0,...,False,False,False,False,False,False,False,False,False,True
2,2008-12-04,14.1,20.6,7.8,7.0,7.2,17.0,13.0,41.0,69.0,...,False,False,False,False,False,False,False,False,False,True
3,2008-12-05,13.0,17.8,0.0,3.6,4.9,17.0,13.0,19.0,73.0,...,False,False,False,False,False,False,True,False,False,False
4,2008-12-06,14.8,18.5,0.0,3.8,8.7,17.0,11.0,20.0,46.0,...,False,False,False,False,False,False,False,False,False,False


#### 1.3. Separe el dataset en entrenamiento y prueba (70% entrenamiento, 30% prueba).

In [7]:
# Separamos variables predictoras de variables a predecir

x = df_encoded.drop(columns="RainTomorrow")
y = df_encoded["RainTomorrow"]

In [8]:
# Separamos el dataset en train y test

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

print(f"El tamaño de entrenamiento es: {x_train.shape}")
print(f"El tamaño de prueba es: {x_test.shape}")

El tamaño de entrenamiento es: (75686, 109)
El tamaño de prueba es: (32437, 109)


In [9]:
x_train.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
51307,2012-08-10,3.8,14.6,0.0,4.0,10.3,52.0,26.0,35.0,65.0,...,False,False,False,False,False,False,True,False,False,False
64365,2016-03-16,14.2,31.2,0.0,7.6,11.2,31.0,7.0,13.0,39.0,...,False,False,False,False,False,False,False,False,False,False
94139,2010-05-30,10.8,14.5,7.2,0.2,1.0,22.0,11.0,7.0,97.0,...,False,False,False,False,False,False,False,False,True,False
23601,2014-07-08,-3.9,11.2,0.0,1.6,2.8,37.0,6.0,26.0,99.0,...,True,False,False,False,False,False,False,False,False,False
10245,2012-04-12,1.6,20.8,0.0,5.8,11.0,24.0,15.0,13.0,85.0,...,False,True,False,False,False,False,False,False,False,False


#### 1.4. Normalice o estandarice los atributos numéricos para asegurar que todas las variables tengan la misma escala.

In [10]:
# Retiramos la variable Date

x_train.drop(columns="Date", inplace=True)
x_test.drop(columns="Date", inplace=True)

x_test.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
104335,16.8,25.0,9.8,47.0,7.4,41.0,0.0,31.0,88.0,78.0,...,False,False,False,False,False,False,False,False,False,False
20196,22.7,28.4,1.0,6.6,6.7,59.0,28.0,35.0,67.0,65.0,...,False,False,False,True,False,False,False,False,False,False
20950,19.0,28.0,0.0,26.4,8.1,31.0,15.0,19.0,73.0,63.0,...,False,False,False,False,False,False,False,False,False,False
95649,3.7,18.8,3.6,1.4,11.9,35.0,7.0,20.0,59.0,34.0,...,False,False,False,False,False,True,False,False,False,False
23809,11.3,25.2,0.2,1.6,2.8,48.0,9.0,7.0,70.0,39.0,...,False,False,False,False,False,False,True,False,False,False


In [11]:
# Convertimos las bool en entero

col_bool = x_train.select_dtypes(include=['bool']).columns
col_num = x_train.select_dtypes(include=['float64']).columns

x_train[col_bool] = x_train[col_bool].astype(int)
x_test[col_bool] = x_test[col_bool].astype(int)

x_test.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
104335,16.8,25.0,9.8,47.0,7.4,41.0,0.0,31.0,88.0,78.0,...,0,0,0,0,0,0,0,0,0,0
20196,22.7,28.4,1.0,6.6,6.7,59.0,28.0,35.0,67.0,65.0,...,0,0,0,1,0,0,0,0,0,0
20950,19.0,28.0,0.0,26.4,8.1,31.0,15.0,19.0,73.0,63.0,...,0,0,0,0,0,0,0,0,0,0
95649,3.7,18.8,3.6,1.4,11.9,35.0,7.0,20.0,59.0,34.0,...,0,0,0,0,0,1,0,0,0,0
23809,11.3,25.2,0.2,1.6,2.8,48.0,9.0,7.0,70.0,39.0,...,0,0,0,0,0,0,1,0,0,0


In [12]:
from sklearn.preprocessing import StandardScaler

# Se estandarizan los atributos numéricos
scW = StandardScaler()

x_train_transformed = scW.fit_transform(x_train)
x_test_transformed = scW.transform(x_test)

In [13]:
x_test_transformed

array([[ 0.64630459,  0.18108769,  0.85948031, ..., -0.27048938,
        -0.25391093, -0.2625563 ],
       [ 1.55956415,  0.66082663, -0.15094816, ..., -0.27048938,
        -0.25391093, -0.2625563 ],
       [ 0.98684205,  0.60438676, -0.26576957, ..., -0.27048938,
        -0.25391093, -0.2625563 ],
       ...,
       [-0.76228218, -1.35689892,  0.4690875 , ..., -0.27048938,
        -0.25391093, -0.2625563 ],
       [ 0.41411996, -0.66551044, -0.24280529, ..., -0.27048938,
        -0.25391093, -0.2625563 ],
       [-0.29791291, -0.80661013,  4.57969423, ..., -0.27048938,
        -0.25391093, -0.2625563 ]])

### 2. Modelado y Evaluación:

#### 2.1. Determine dos métricas de evaluación que considere importante medir para este problema. Justifique su elección.

In [14]:
# La métrica esta balanceada:

distribucion_clase = y_train.value_counts(normalize=True)
distribucion_clase

RainTomorrow
False    0.782311
True     0.217689
Name: proportion, dtype: float64

por lo cual, las clases estan desbalanceadas

Dado que la data se muestra que las clases no son balanceadas, se usarán las siguientes métricas:

* Exactitud balanceada: Se elige para identificar si el clasificador es bueno prediciendo.

* Puntaje F1: Elegimos esta métrica debido a que las clases estan desquilibradas y los falsos positivos como falsos negativos son importantes.

#### 2.2. Implemente un modelo de base usando como predicción que determine que llueve aquí si hoy llueve, y si hoy no llueve mañana no va a llover.

In [15]:
# Implementado el modelo base

from sklearn.metrics import balanced_accuracy_score, f1_score

prediccion = df["RainToday"]

# Calculamos la precisión del modelo
exactitud = balanced_accuracy_score(df["RainTomorrow"], prediccion)

# Calculemos el F1 score
f1 = f1_score(df["RainTomorrow"], prediccion)

print(f"La exactitud del modelo base es: {exactitud*100:.2f}%")
print(f"El F1-score del modelo base es: {f1*100:.2f}%")

La exactitud del modelo base es: 65.47%
El F1-score del modelo base es: 46.00%


#### 2.3. Implemente y entrene un modelo de k Nearest Neighbors (kNN) para predecir si va a llover mañana. Utilice la distancia euclidiana como medida de distancia entre vecinos. Ajuste el parámetro k utilizando técnicas como la validación cruzada (5 folds) para optimizar el rendimiento del modelo (utilice como función de optimización una de las métricas definidas en el punto anterior).

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# definimos modelo y parámetros
knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': range(1, 21),
    'metric': ['euclidean']
}

# definimos la métrica F1
scorer = make_scorer(balanced_accuracy_score)

# se realiza la búsqueda con validación cruzada
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring=scorer)
grid_search.fit(x_train_transformed, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean'], 'n_neighbors': range(1, 21)},
             scoring=make_scorer(balanced_accuracy_score, response_method='predict'))

In [17]:
# Obtenemos los mejores hiperparámetros y el mejor score

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores hiperparámetros encontrados:")
print(best_params)
print("Mejor score de validación cruzada obtenido: {:.2f}".format(best_score))

Mejores hiperparámetros encontrados:
{'metric': 'euclidean', 'n_neighbors': 1}
Mejor score de validación cruzada obtenido: 0.64


In [18]:
# Modelo a evaluar

classifier_best_knn = KNeighborsClassifier(**best_params)
classifier_best_knn.fit(x_train_transformed, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=1)

In [19]:
# Predicción

y_best_pred_train = classifier_best_knn.predict(x_train_transformed)
y_best_pred_test = classifier_best_knn.predict(x_test_transformed)

In [20]:
# Exactitud del modelo

acc_test = balanced_accuracy_score(y_test, y_best_pred_test)

print(f"La exactitud del modelo es: {acc_test*100:.2f}%")

La exactitud del modelo es: 64.21%


In [21]:
# F1-score del modelo

f1_score_test = f1_score(y_test, y_best_pred_test)

print(f"El F1-score del modelo es: {f1_score_test*100:.2f}%")

El F1-score del modelo es: 44.10%


#### 2.4. Implemente y entrene un modelo de regresión logística para predecir la misma variable objetivo.

In [22]:
from sklearn.linear_model import LogisticRegression

log_reg_rain_model = LogisticRegression(random_state=42)
log_reg_rain_model.fit(x_train_transformed, y_train)

LogisticRegression(random_state=42)

In [23]:
# Prediciendo

y_pred_test = log_reg_rain_model.predict(x_test_transformed)

In [24]:
# Exactitud del modelo

reg_acc_test = balanced_accuracy_score(y_test, y_pred_test)

print(f"La exactitud del modelo es:{reg_acc_test*100:.2f}%")

La exactitud del modelo es:72.52%


In [25]:
# El F1-score del modelo

reg_f1_test = f1_score(y_test, y_pred_test)

print(f"El F1-score del modelo es: {reg_f1_test*100:.2f}%")

El F1-score del modelo es: 59.46%


#### 2.5. Evaluar los dos modelos con las métricas definidas al principio. Discuta las fortalezas y debilidades de cada enfoque en el contexto específico de este problema comparando con el modelo de base.

##### Modelo base:

- Precisión balanceada: 65.47%
- F1-score: 46%

El modelo de base proporciona un rendimiento sorprendentemente bueno dada su simplicidad. Sin embargo, su F1-Score indica que no maneja bien los falsos positivos y falsos negativos, lo que puede ser un problema.

##### KNN:

- Precisión balanceada: 64.21%
- F1-score: 44.10%

El modelo KNN no logró superar al modelo base ni en precisión balanceada ni en F1-Score. Esto podría deberse a la sensibilidad del KNN al ruido y a la naturaleza del problema meteorológico, donde puede que no siempre sea fácil identificar vecinos cercanos que ayuden a mejorar las predicciones.

###### Fortalezas:

- KNN es fácil de interpretar y entender cómo funciona.

###### Debilidades:

- La precisión balanceada del KNN (64.21%) es ligeramente inferior a la del modelo base. Esto podría deberse a que KNN es sensible a la escala de los datos y podría no estar capturando patrones complejos en los datos meteorológicos.
- Con un F1-Score de 44.10%, es aún menos eficaz que el modelo base en capturar la precisión y la sensibilidad.
- KNN puede ser muy sensible a puntos de datos atípicos o ruido en el conjunto de datos, lo que podría afectar su rendimiento.

##### Regresión Logística:

- Precisión balanceada: 72.52%
- F1-Score: 59.46%

La regresión logística muestra un claro mejor rendimiento tanto en precisión balanceada como en F1-Score. Esto sugiere que el modelo es más capaz de capturar patrones en los datos meteorológicos y manejar el desbalance de clases mejor que el modelo base y KNN.

###### Fortalezas:

- La precisión balanceada del 72.52% es superior a la del modelo base y KNN, lo que sugiere que la regresión logística está capturando patrones más complejos en los datos. También tiene el F1-Score más alto (59.46%), lo que indica que maneja mejor tanto la precisión como la sensibilidad.
- Con un mejor F1-Score, la regresión logística es más efectiva en manejar el desbalance de clases.

###### Debilidades:

- Aunque la regresión logística es un modelo robusto, puede verse limitada si los datos subyacentes tienen relaciones no lineales más complejas.


### 3. Conclusiones:

#### 3.1. Resuma los hallazgos más relevantes obtenidos.

- La regresión logística es el modelo más efectivo en este contexto, superando tanto al modelo base como al modelo KNN en términos de precisión balanceada y F1-Score.
- Si bien el modelo base es sorprendentemente competitivo, su simplicidad lo limita.
- KNN no ofrece mejoras significativas, lo que sugiere que un enfoque lineal como la regresión logística es más adecuado para este problema de predicción de lluvia.
- Aunque en teoría KNN debería ser más poderoso que el modelo base, en este caso no lo supera. KNN podría requerir un mayor ajuste de hiperparámetros y técnicas más avanzadas de selección de características.

#### 3.2. Guarde los conjuntos de entrenamiento y evaluación para el siguiente TP en archivos `csv`.

In [26]:
# Guardando los conjuntos de datos
features = x.columns.drop('Date')

# crear los dataframes
x_train_df = pd.DataFrame(x_train_transformed, columns=features)
x_test_df = pd.DataFrame(x_test_transformed, columns=features)
y_train_df = pd.DataFrame(y_train, columns=['RainTomorrow'])
y_test_df = pd.DataFrame(y_test, columns=['RainTomorrow'])

x_train_df.to_csv('x_train.csv', index = False)
x_test_df.to_csv('x_test.csv', index = False)
y_train_df.to_csv('y_train.csv', index=False)
y_test_df.to_csv('y_test.csv', index=False)

print("Conjuntos guardados exitosamente")

Conjuntos guardados exitosamente


#### 3.3. Guarde en formato pickle el mejor modelo, seleccionado por su criterio.

In [27]:
import pickle

with open('best_model.pkl', 'wb') as file:
    pickle.dump(log_reg_rain_model, file)

print("El modelo ha sido guardado satisfactoriamente")

El modelo ha sido guardado satisfactoriamente
